<a href="https://colab.research.google.com/github/isabellahg/map-reduce/blob/main/2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

!wget https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar -xf spark-3.2.3-bin-hadoop3.2.tgz
!pip install -q findspark


## Iniciar sesión

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Spark_Dataframes")\
        .getOrCreate()

# crear contexto de spark
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

--2023-02-06 18:45:05--  https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301136158 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.3-bin-hadoop3.2.tgz.1’

spark-3.2.3-bin-had 100%[===================>] 287.19M   175MB/s    in 1.6s    

2023-02-06 18:45:07 (175 MB/s) - ‘spark-3.2.3-bin-hadoop3.2.tgz.1’ saved [301136158/301136158]



In [3]:
## Utilizar esta celda SOLO si el dataset lo tenemos en Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

ds = spark.read.csv('/content/gdrive/MyDrive/Colab Notebooks/weblog.csv', header=True)
weblog_rdd = ds.rdd

Mounted at /content/gdrive


## A

In [4]:
urls = (weblog_rdd
             .map(lambda item: (item.URL, 1))
             .reduceByKey(lambda x,y: x+y))
urls.collect()

[('GET /login.php HTTP/1.1', 3284),
 ('POST /process.php HTTP/1.1', 312),
 ('GET /home.php HTTP/1.1', 2640),
 ('GET /js/vendor/moment.min.js HTTP/1.1', 173),
 ('GET /bootstrap-3.3.7/js/bootstrap.js HTTP/1.1', 191),
 ('GET /profile.php?user=bala HTTP/1.1', 1),
 ('GET /js/jquery.min.js HTTP/1.1', 56),
 ('GET /js/chart.min.js HTTP/1.1', 58),
 ('GET /edit.php?name=bala HTTP/1.1', 1),
 ('GET /logout.php HTTP/1.1', 42),
 ('GET / HTTP/1.1', 861),
 ('GET /contestproblem.php?name=RUET%20OJ%20Server%20Testing%20Contest HTTP/1.1',
  467),
 ('GET /css/bootstrap.min.css HTTP/1.1', 404),
 ('GET /css/font-awesome.min.css HTTP/1.1', 399),
 ('GET /css/normalize.css HTTP/1.1', 408),
 ('GET /css/style.css HTTP/1.1', 395),
 ('GET /js/vendor/modernizr-2.8.3.min.js HTTP/1.1', 1415),
 ('GET /css/main.css HTTP/1.1', 394),
 ('GET /js/vendor/jquery-1.12.0.min.js HTTP/1.1', 387),
 ('GET /bootstrap-3.3.7/js/bootstrap.min.js HTTP/1.1', 382),
 ('GET /fonts/fontawesome-webfont.woff2?v=4.6.3 HTTP/1.1', 245),
 ('GET /

## B

In [5]:
unique_access = (weblog_rdd
                  .filter(lambda item: item.Staus == '200' and 'php' in item.URL)
                  .map(lambda item: (item.IP, item.URL))
                  .distinct()
                  .map(lambda item: (item[1], 1))
                  .reduceByKey(lambda x,y: x + y)
                  )
unique_access.collect()

[('GET /login.php HTTP/1.1', 5),
 ('GET /home.php HTTP/1.1', 5),
 ('GET /profile.php?user=bala HTTP/1.1', 1),
 ('GET /edit.php?name=bala HTTP/1.1', 1),
 ('GET /contestproblem.php?name=RUET%20OJ%20Server%20Testing%20Contest HTTP/1.1',
  5),
 ('GET /login.php?value=fail HTTP/1.1', 5),
 ('GET /countdown.php?name=RUET%20OJ%20Server%20Testing%20Contest HTTP/1.1',
  5),
 ('GET /compiler.php HTTP/1.1', 5),
 ('GET /details.php?id=44 HTTP/1.1', 5),
 ('GET /contest.php HTTP/1.1', 5),
 ('GET /details.php?id=46 HTTP/1.1', 5),
 ('GET /details.php?id=42 HTTP/1.1', 5),
 ('GET /details.php?id=45 HTTP/1.1', 5),
 ('GET /contestproblem.php?name=RUET%20OJ%20TLE%20Testing%20Contest HTTP/1.1',
  5),
 ('GET /archive.php HTTP/1.1', 5),
 ('GET /allsubmission.php HTTP/1.1', 5),
 ('GET /showcode.php?id=284&nm=xyz_xyz HTTP/1.1', 4),
 ('GET /showcode.php?id=281&nm=shawon HTTP/1.1', 4),
 ('GET /index.php HTTP/1.1', 1),
 ('GET /contestproblem.php?name=Toph%20Contest%202 HTTP/1.1', 1),
 ('GET /contestproblem.php?name

## C

In [6]:
urls_by_client = (weblog_rdd
                  .map(lambda item: (item.IP, item.URL))
                  .distinct()
                  .map(lambda item: (item[0], 1))
                  .reduceByKey(lambda x,y: x + y)
)
urls_by_client.collect()

[('10.128.2.1', 149),
 ('10.131.2.1', 124),
 ('10.130.2.1', 135),
 ('10.129.2.1', 127),
 ('10.131.0.1', 155),
 ('chmod:', 2),
 ('rm:', 2),
 ('timeout:', 1),
 ('sh:', 6),
 ('a.out:', 1),
 ('[Tue', 5),
 ('[Wed', 1),
 ('[Thu', 2),
 ('[Fri', 1),
 ('[Sat', 1),
 ('[Mon', 1)]

## D

In [7]:
access_freq = (weblog_rdd
               .map(lambda item: ((item.IP, item.URL), 1))
               .reduceByKey(lambda x,y: x+y))
access_freq.collect()

[(('10.128.2.1', 'GET /login.php HTTP/1.1'), 991),
 (('10.128.2.1', 'POST /process.php HTTP/1.1'), 65),
 (('10.128.2.1', 'GET /home.php HTTP/1.1'), 872),
 (('10.131.2.1', 'GET /js/vendor/moment.min.js HTTP/1.1'), 23),
 (('10.130.2.1', 'GET /bootstrap-3.3.7/js/bootstrap.js HTTP/1.1'), 55),
 (('10.130.2.1', 'GET /profile.php?user=bala HTTP/1.1'), 1),
 (('10.128.2.1', 'GET /js/jquery.min.js HTTP/1.1'), 16),
 (('10.131.2.1', 'GET /js/chart.min.js HTTP/1.1'), 11),
 (('10.131.2.1', 'GET /edit.php?name=bala HTTP/1.1'), 1),
 (('10.131.2.1', 'GET /logout.php HTTP/1.1'), 9),
 (('10.131.2.1', 'GET /login.php HTTP/1.1'), 173),
 (('10.130.2.1', 'GET /login.php HTTP/1.1'), 959),
 (('10.130.2.1', 'GET / HTTP/1.1'), 167),
 (('10.129.2.1', 'POST /process.php HTTP/1.1'), 44),
 (('10.131.0.1', 'GET /home.php HTTP/1.1'), 788),
 (('10.131.0.1',
   'GET /contestproblem.php?name=RUET%20OJ%20Server%20Testing%20Contest HTTP/1.1'),
  100),
 (('10.131.2.1', 'GET /css/bootstrap.min.css HTTP/1.1'), 67),
 (('10.128

## E


In [8]:
wrong_responses = (weblog_rdd
               .filter(lambda item: item.Staus != '200') 
               .map(lambda item: (item.IP, item.Staus))
               .distinct()
               .map(lambda item: (item[0], [item[1]]))
               .reduceByKey(lambda x,y: x + y)
)
wrong_responses.collect()

[('10.128.2.1', ['302', '304', '404', '206']),
 ('10.131.2.1', ['302', '304', '206', '404']),
 ('10.130.2.1', ['302', '304', '404', '206']),
 ('10.129.2.1', ['302', '304', '206', '404']),
 ('10.131.0.1', ['304', '302', '206', '404']),
 ('chmod:', ['No']),
 ('rm:', ['No']),
 ('timeout:', ['dumped']),
 ('sh:', ['Segmentation', 'Aborted', 'found']),
 ('a.out:', ['Assertion']),
 ('[Tue', ['2017]', '2018]']),
 ('[Wed', ['2017]']),
 ('[Thu', ['2017]', '2018]']),
 ('[Fri', ['2018]']),
 ('[Sat', ['2018]']),
 ('[Mon', ['2018]'])]